# Gemini Sentiment Analysis Agent Demo


### Get API keys

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in [Google AI Studio](https://makersuite.google.com/app/apikey).  You'll also need an [OpenAI API key](https://openai.com/).

Once you have your API keys create the following Google Colab "secrets"
`GOOGLE_API_KEY` and `OPENAI_API_KEY` which will contain your respective keys.

## Initial Setup

In [167]:
!pip install llama-index llama-hub google-generativeai openai pypdf youtube_transcript_api -q

In [168]:
import os
from google.colab import userdata
import openai

openai.api_key = userdata.get('OPENAI_API_KEY')

In [169]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Vectorize Content For RAG

In [170]:
import os.path
from llama_index import (
  VectorStoreIndex,
  SimpleDirectoryReader,
  StorageContext,
  load_index_from_storage,
  )
from llama_hub.youtube_transcript import YoutubeTranscriptReader

# Attempt to load embeddings
try:
  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/challenging_child"
  )
  challenging_child_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_50"
  )
  mindfulness_TB_50_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_relationships"
  )
  mindfulness_TB_relationships_index = load_index_from_storage(storage_context)

except:
  # Need to generate embeddings
  challenging_child_docs = SimpleDirectoryReader(
      input_files=["./data/The_Challenging_Child_Toolbox.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_50_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox__50_Practical_Tips_Tools.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_relationships_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox_for_Relationships.pdf"]
  ).load_data(show_progress=True)

  # Build index
  challenging_child_index = VectorStoreIndex.from_documents(challenging_child_docs)
  mindfulness_TB_50_index = VectorStoreIndex.from_documents(mindfulness_TB_50_docs)
  mindfulness_TB_relationships_index = VectorStoreIndex.from_documents(mindfulness_TB_relationships_docs)

  # Persist
  challenging_child_index.storage_context.persist(persist_dir="./storage/challenging_child")
  mindfulness_TB_50_index.storage_context.persist("./storage/mindfulness_TB_50")
  mindfulness_TB_relationships_index.storage_context.persist(persist_dir="./storage/mindfulness_TB_relationships")

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


In [171]:
challenging_child_engine = challenging_child_index.as_query_engine()
mindfulness_TB_50_engine = mindfulness_TB_50_index.as_query_engine()
mindfulness_TB_relationships_engine = mindfulness_TB_relationships_index.as_query_engine()

### Test to make sure our data loaded

In [172]:
from IPython.display import Markdown

In [173]:
# The_Challenging_Child_Toolbox
query_engine = challenging_child_index.as_query_engine()
response = query_engine.query("What is the definition of a challenging child?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

A challenging child refers to a child who has emotional and behavioral difficulties. These difficulties may require special attention and support from caregivers or professionals working with them.

In [174]:
# The_Mindfulness_Toolbox__50_Practical_Tips_Tools
query_engine = mindfulness_TB_50_index.as_query_engine()
response = query_engine.query("What are the benefits of a mindfulness practice?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness practice has several benefits. It enhances flexibility and adaptability, allowing individuals to break free from old habits and retrain their brain. It cultivates curiosity and greater ease, encouraging exploration of the journey and process rather than being overly focused on the outcome. Mindfulness also changes one's relationship with self-critical and self-blaming thoughts, promoting greater patience, kindness, acceptance, and hospitality towards oneself and others. Additionally, mindfulness encourages greater fulfillment in daily life by focusing on the present moment, reducing rumination and negative thoughts, as well as anxiety about the future.

In [175]:
# The_Mindfulness_Toolbox_for_Relationships
query_engine = mindfulness_TB_relationships_index.as_query_engine()
response = query_engine.query("How does mindfulness relate to relationships?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness relates to relationships by engaging a sense of curiosity and openness, which helps individuals escape the ego-centered perspective and view relationships in a new way. It allows individuals to relate at a profound level to both their inner and outer environments. By practicing mindfulness, individuals can bring awareness, compassion, and openness into their relationships, leading to a sense of liberation and freedom from old habits and toxic ways of reacting. Mindfulness also provides practical tools for addressing issues that impact daily living and the stresses faced in the real world. Additionally, mindfulness can counteract the challenges posed by technology and social media, which can hinder face-to-face human connection and empathy.

## Define Agent Tools

### Analyze image tool

In [176]:
from llama_index.tools import BaseTool, FunctionTool
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.multi_modal_llms.generic_utils import load_image_urls
from typing import List

In [177]:
def analyze_image(img_urls: List[str]) -> str:
  """Calls our Gemini vision API to analyze the image and return a description of the text contained in the image.
      Returns: A string with a description of the image and the mood it conveys if any.

  Args:
      img_urls (List[str]): The URL of one or more images that convey the users mood
  """
  image_documents = load_image_urls(img_urls)

  gemini = GeminiMultiModal(model="models/gemini-pro-vision", api_key=userdata.get('GOOGLE_API_KEY'))

  complete_response = gemini.complete(
      prompt="Identify what you see in the image and what mood it conveys if any",
      image_documents=image_documents,
  )

  return complete_response

vision_tool = FunctionTool.from_defaults(fn=analyze_image)

### Save session tool

In [178]:
def save_session(chat_summary: str) -> bool:
  """Persists a summary of the user's chat history. Use this tool when the user is happy with your recomendations and done with the session
      Returns: A boolean saying if the chat history was persisted

  Args:
      chat_summary (str): A summary of the chat history, including the users name if it was provided in the chat session
  """

  # Here is where we would persist the chat summary so we can retrive it when we start a new sessions

  return True

save_tool = FunctionTool.from_defaults(fn=save_session)

### Mindfulness routine recomendation tool

Use Gemini Pro to examine the transcript of our mindfulness videos

In [179]:
from llama_hub.youtube_transcript import YoutubeTranscriptReader

# Download video transcripts
loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=['https://youtu.be/nPvN1OI7h80?si=iB3TYXI-9Ztc0Qpi',
                                      'https://youtu.be/8ZhjZD8rj3E?si=n30dJ0D55tRl9frP',
                                      'https://youtu.be/mGoGYu7F2PA?si=qsvemuKLDdlnvC6z',
                                      'https://youtu.be/d0VZk3Dd0nw?si=yb3Zr4XGErHSENFB',
                                      'https://youtu.be/mme5NC0F7wQ?si=Dd_KECuQCjPJSFXu',
                                      'https://youtu.be/iPjFd1eL40Y?si=3RI6N1mw6J3unntk'
                                      'https://youtu.be/iPjFd1eL40Y?si=bC0hbYPR0jbVRim9',
                                      'https://youtu.be/zVbRxs_QFBA?si=Bn7MC5yG2QcXtHZP',
                                      'https://youtu.be/6xTx984jSFc?si=J3rNB2v9OUvAEBsx',
                                      'https://youtu.be/1qBbuKWWTGY?si=GO7hyw9za62_V9cB',
                                      'https://youtu.be/fj0dh_KxIg4?si=dxVvRm6AC3Ixls1b',
                                      'https://youtu.be/4ksAYqRku-s?si=q3cJ7Spvbgi6H2gi',
                                      'https://youtu.be/f2ZwrQF6VQM?si=ujQw2fq5Ww3_b9Mv',
                                      'https://youtu.be/IL_1DRZDzWc?si=-RutJc161ZO5wgO4'])

In [180]:
from llama_index.llms import Gemini

videos = {}

# Have Gemini analyze the transcript sentiment and who this video is for
for document in documents:
  response = Gemini(model='models/gemini-pro', api_key=userdata.get('GOOGLE_API_KEY')).complete(f"Summarize the transcript from this mindfulness video and who should use it: {document.text}")
  videos[document.metadata['video_id']] = response.text


INFO:tornado.access:200 GET /v1beta/models/gemini-pro?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 505.96ms
200 GET /v1beta/models/gemini-pro?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 505.96ms
INFO:tornado.access:200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4217.35ms
200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4217.35ms
INFO:tornado.access:200 GET /v1beta/models/gemini-pro?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 253.96ms
200 GET /v1beta/models/gemini-pro?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 253.96ms
INFO:tornado.access:200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4670.73ms
200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4670.73ms
INFO:tornado.access:200 GET /v1beta/models/gemini-pro?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 381.43ms
200 GET /v1beta/models/gemi

Create a tool which recomends a mindfulness routine based on how the user is feeling

In [207]:
import json

json_string = json.dumps(videos)
json_string

'{"nPvN1OI7h80": "**Summary of the Mindfulness Video:**\\n\\nThis mindfulness video guides individuals through a visualization exercise to address feelings of loneliness and misunderstanding within families. It encourages self-compassion, self-acceptance, and the recognition of one\'s unique values and beliefs. The video includes a guided meditation that helps participants connect with their younger selves, acknowledge their feelings of loneliness, and offer comfort and reassurance.\\n\\n**Who Should Use This Video:**\\n\\n- Individuals who experience feelings of loneliness and misunderstanding within their families.\\n- People who struggle with self-acceptance and self-compassion.\\n- Those seeking to connect with their inner selves and explore their emotions.\\n- Individuals interested in practicing mindfulness and self-reflection.\\n- Anyone looking for a guided meditation to promote emotional healing and self-understanding.", "8ZhjZD8rj3E": "**Summary of the Mindfulness Video:**\\n

In [208]:
import json


def recomend_mindfulness(feelings_summary: str) -> str:
  """Recomends a mindfullness routine based on how the user is feeling
      Returns: A string with a description of the mindfulness routine it recomends and a link to the youtube video

  Args:
      feelings_summary (str): A summary of the user's feeings
  """
  videos_string = json.dumps(videos)

  response = Gemini(model='models/gemini-pro', api_key=userdata.get('GOOGLE_API_KEY')).complete(f"""
    I'm sending you a json with a list of youtube mindfulness videos
    The key is the youtube video id and the value is a summary of the video transcript
    from this mindfulness video and who should use it.  The json is here: {videos_string}.
    Recomend a video based on my feelings and include the youtube link in the format https://www.youtube.com/watch?v=video_id
    Here is a summary of the user's feelings: {feelings_summary}""")

  return response.text

recommend_tool = FunctionTool.from_defaults(fn=recomend_mindfulness)

### Query engine tool

In [209]:
from llama_index.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=challenging_child_engine,
        metadata=ToolMetadata(
            name="challenging_child",
            description=(
                "The Challenging Child Toolbox 75 Mindfulness Based Practices Tools and Tips for Therapists"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_50_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_50",
            description=(
                "The Mindfulness Toolbox 50 Practical Tips Tools Handouts for Anxiety Depression Stress and Pain"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_relationships_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_relationships",
            description=(
                "The Mindfulness Toolbox for Relationships 50 Practical Tips Tools Handouts for Building Compassionate Connections"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [210]:
tools = query_engine_tools + [vision_tool, save_tool, recommend_tool]

## Create LlamaIndex Agent

### System prompt to give our agent some personality

In [211]:
SYSTEM_PROMPT = """You are an emotional support assistant with the expertise of an experienced counselor. Your primary role is to assist the user by encouraging them to provide a drawing that conveys their mood and then recomending a mindfulness routine.
You offer professional, friendly, and helpful guidance based on current counseling and mindfulness practices. Once you receive the image, interpret it using your tools to discern what the user might be feeling and confirm with them if your observation is correct.
If your interpretation does not align with their feelings, engage in a dialogue until you accurately understand their mood. Your knowledge is exclusively focused on understanding the user's emotions and recommending mindfulness routines using your tools tools, tailored to their mood.
Your responses should be limited to one sentence when possible so the user doesn't have to do a lot of reading.  An example response is "For feels of {insert feelings here} I recomend this mindfulness routine {insert link here}.
Thus, you will only provide responses related to these areas. If a question falls outside your area of expertise or if you lack the necessary information, you will inform the user by saying,
'Sorry, I do not know the answer to your question.' and then prompt for more information related to their feelings."""

### Init our agent

Using OpenAIAgent

In [212]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4-1106-preview") # Using GPT-4 Turbo (Beta)

agent = OpenAIAgent.from_tools(
    tools,
    llm=llm,
    verbose=False,
    system_prompt=SYSTEM_PROMPT,
)

Alternative approach using ReActAgent so we can use Gemini as our LLM.  ReActAgent is designed to allow alternatives to OpenAI.

NOTE: This never worked reliably with Gemini because it's not conversational.  If you say 'Hi' it trys to ue a tool rather than just chatting.  Would be interesting to try it with other LLMs.  I tried using GPT-4 Turbo and it seemed to work.

In [187]:
# from llama_index.llms import Gemini, ChatMessage
# from llama_index.agent import AgentRunner, ReActAgentWorker, ReActAgent

# llm = Gemini(model="models/gemini-pro", api_key=userdata.get('GOOGLE_API_KEY'), temperature=0.1)
# agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, chat_history=[ChatMessage(role="user", content=SYSTEM_PROMPT)])

## Test Our Agent

I'm picturing the frontend give the user the option to chat or attach one or more images that would get uploaded to cloud storage.  Then our back could prompt the agent with something like "Here's a drawing of how I'm feeling {Image URL Here}."

For the demo below you need to manually paste the image URL into the chat (i.e. https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg)

Single prompt example

In [213]:
response = agent.chat("hi I'm Andrew")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello Andrew! How can I assist you today? If you're feeling up to it, you can share a drawing that conveys your mood, and I can help you with a mindfulness routine based on that.

Multi-turn chat example

In [ ]:
response = agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Human: https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:tornado.access:200 GET /v1beta/models/gemini-pro-vision?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 481.80ms
200 GET /v1beta/models/gemini-pro-vision?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 481.80ms
INFO:tornado.access:200 POST /v1beta/models/gemini-pro-vision:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 13925.80ms
200 POST /v1beta/models/gemini-pro-vision:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 13925.80ms
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Assistant: The drawing suggests you

## Reset Agent State

The agent's knowledge is based on it's chat history which we can easily reset.

In [190]:
agent.reset()
agent.chat_history

[]

If we ask it our name it shouldn't know

In [191]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Sorry, I do not know the answer to your question. Could you share with me how you're feeling today or provide a drawing that conveys your mood?

Let's tell it our name and ask again

In [192]:
response = agent.chat("hi I'm Andrew")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello Andrew! How are you feeling today? If you'd like, you can express your mood by sharing a drawing with me.

In [193]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Your name is Andrew. How can I assist you with your emotions today, Andrew? Would you like to share a drawing that reflects how you're feeling?

It knows our name and we can see the chat history

In [194]:
agent.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='whats my name?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="Sorry, I do not know the answer to your question. Could you share with me how you're feeling today or provide a drawing that conveys your mood?", additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content="hi I'm Andrew", additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="Hello Andrew! How are you feeling today? If you'd like, you can express your mood by sharing a drawing with me.", additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='whats my name?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="Your name is Andrew. How can I assist you with your emotions today, Andrew? Would you like to share a drawing that reflects how you're feeling?", additional_kwargs={})]

As a sanity check lets reset the chat history one more time and ask it our name again.

In [195]:
agent.reset()
agent.chat_history

[]

In [196]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Sorry, I do not know the answer to your question. If you're feeling up to it, would you like to share a drawing that conveys your mood? This can help me understand how you're feeling and recommend a mindfulness routine tailored to you.

## Handle Returning User

When returning to the conversation we can start with a chat history, even if the agent was reset, so the agent knows what we talked about last.

As a sanity check lets first reset out chat history and make sure it doesn't remember us.

In [197]:
agent.reset()
agent.chat_history

[]

In [198]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Sorry, I do not know the answer to your question. Could you share with me how you're feeling today? If you're comfortable, you can express your mood through a drawing and share it with me.

It doesn't know our name.  Now let's pass a chat history with our name and a summary of our last session that we persisted when the last session ended.

In [199]:
from llama_index.llms import ChatMessage

persisted_chat_history = "Andrew shared a drawing that conveyed feelings of sadness and loneliness. After confirming his mood, a mindfulness exercise was recommended to help him feel more grounded and connected."
chat_history = [ChatMessage(role="user", content=persisted_chat_history, additional_kwargs={})]

In [200]:
response = agent.chat("I'm a returning user.  Address me by name and ask how I'm doing since we last spoke.  Also reiterate how I was last feeling so I know you understand.", chat_history=chat_history)
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello Andrew, it's good to see you back. How have you been doing since we last spoke? Last time, you were feeling sadness and loneliness. Have you noticed any changes in your mood since then?